# Spanish Word Embeddings

# This module implements word vectors and their similarity look-ups.

we use gesim package to perform a wormbeddings (word2vec) transformation in order we can compare the users text and company textsuch that a match can be done between them 

In [7]:
import numpy as np
import pandas as pd
import math
from numpy import dot
from numpy.linalg import norm
from sklearn.metrics.pairwise import cosine_similarity

In [1]:
#installing gensim
#import sys
#!conda install -c conda-forge gensim --yes

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Anaconda

  added / updated specs:
    - gensim


The following packages will be UPDATED:

  gensim             pkgs/main::gensim-3.4.0-py36hfa6e2cd_0 --> conda-forge::gensim-3.7.3-py36h6538335_0


Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done


# LOADING THE WORD2VEC MODEL

loading a pre-trained spanish word-embedding, with a vector dim of 300 and 10000 vectors -> for more infortion see the web site : https://github.com/dccuchile/spanish-word-embeddings

In [3]:
from gensim.models.keyedvectors import KeyedVectors
wordvectors_file_vec = 'fasttext-sbwc.3.6.e20.vec'
cantidad = 100000
wordvectors = KeyedVectors.load_word2vec_format(wordvectors_file_vec, limit=cantidad)

In [4]:
#example 
vector = wordvectors.get_vector('casa')
print(vector)

[ 1.2100e-01 -1.5552e-01 -3.5114e-02 -1.7998e-01 -1.6266e-01 -1.9180e-01
 -2.5319e-01  1.0108e-01 -1.1969e-01  7.4725e-03 -1.0380e-01  9.8222e-02
  2.8586e-01 -1.2828e-01  2.3204e-01 -1.3754e-03 -1.3301e-01  2.7393e-01
 -3.4402e-02  1.2702e-01 -3.1093e-01 -5.1120e-01 -9.3321e-02 -3.6300e-02
 -6.9176e-02 -1.6083e-01  5.1955e-02 -2.1072e-01 -2.6308e-01 -4.8167e-01
 -4.2471e-01  3.2596e-02 -2.4753e-01  3.7219e-01  3.9991e-02 -4.8988e-02
 -1.2377e-02  1.7868e-01  2.8374e-01  3.5436e-01 -3.9200e-01 -1.3087e-01
 -6.1445e-02  2.0615e-01  4.8538e-01  8.1507e-02 -3.8315e-03  1.1456e-01
  1.9641e-01 -3.1384e-02  1.7718e-01  1.5452e-01  1.2188e-01  1.4018e-01
 -2.2728e-02 -2.0315e-01  4.3390e-02 -3.3416e-01  3.3122e-01 -2.0819e-02
 -7.7117e-02 -4.3856e-02  6.5116e-02  9.3585e-02 -9.1272e-02  1.1774e-01
  1.1442e-01  3.1598e-01  2.7401e-01  4.8852e-03 -1.2554e-01 -7.8936e-02
 -2.8074e-01 -1.9653e-01  8.7350e-02 -2.1922e-01  2.2284e-01 -4.9907e-02
 -5.5925e-01 -3.1795e-01  1.6967e-01  6.9432e-02 -3

#  LOADING THE USERS & COMPANY DATA FOR THE MATCH

In [9]:
# create a pandas dataframe
data_users = pd.read_csv(file1, error_bad_lines=False) 
data_companies = pd.read_csv(file2, error_bad_lines=False) 

In [10]:
data_users = data_users[['user_id','completo']]
#display(data_users).head(10)

In [11]:
demogra_companies = data_companies[['id','name']]
data_companies2 = data_companies[['name','completo_company']]
#display(data_companies2)

# OBTAING THE EMBEDDINGS FOR EACH TEXT IN USERS AND COMPANIES

1) Split the text (user and company) to each word 

2) obtain the word embedding for each word

3) contruct a mean embedding representation for the complete text (user and company)

4) compute the distance between each user mean embedding to each company mean embedding -> the minor distance stablish the macth!

In [12]:
def sentence_to_avg(sentence):
    """
    Converts a sentence (string) into a list of words (strings). Extracts the GloVe representation of each word
    and averages its value into a single vector encoding the meaning of the sentence.
    
    Arguments:
    sentence -- string, one training example from X
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 300-dimensional vector representation
    
    Returns:
    avg -- average vector encoding information about the sentence, numpy-array of shape (300,)
    """
    
    ### START CODE HERE ###
    # Step 1: Split sentence into list of lower case words (≈ 1 line)
    words = sentence.lower().split()

    # Initialize the average word vector, should have the same shape as your word vectors.
    avg = np.zeros(300)
    # Step 2: average the word vectors. You can loop over the words in the list "words".
    total = 0
    for w in words:
        try:
            wv = wordvectors.get_vector(w)
        except:
            continue
        total += 1
        avg = avg + wv
        
    if total != 0:
        avg = avg/total
    #print(total)
    ### END CODE HERE ###
    
    return avg

In [13]:
# EJEMPLO DE USO

# se usa el modelo para obtener el embedding para cada palabra en el texto
vector = sentence_to_avg('la casa esta limpia y es de color azul en la casa hay una niña que canta') #la casa esta limpia y es de color azul en la casa hay una niña que canta
len(vector)

300

# company word-embeddings

In [14]:
# loop over the company text and return the vector
data_companies2['vector_c'] = data_companies2.completo_company.apply(lambda x : sentence_to_avg(x))

display(data_companies2)

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,name,completo_company,vector_c
0,Alianza Team®,consumos masivo alimentos lipidos team solutio...,"[0.051111184392815055, -0.19661288018605877, -..."
1,Sura,digital movilidad salud publica administracion...,"[0.0948477328436503, -0.20946273444824628, -0...."
2,Interacpedia,educacion talento talent pools user experience...,"[0.05852329667229855, -0.2077142979133903, -0...."
3,Bancolombia,emprendedores agro ciberseguridad experiencia ...,"[0.08684147275944656, -0.21992827574623391, -0..."
4,Emtelco,transformacion digital facilitar construccion ...,"[0.08101330395295935, -0.19371718830486126, -0..."
5,Konecta,telecomunicaciones contact center bpo ser comp...,"[0.07819004510540052, -0.21863576104109922, -0..."
6,Satrack INC de Colombia Servisat,monitoreo satelital distribucion logistica acc...,"[0.05171212862149092, -0.2067718740148871, -0...."
7,Auteco Mobility,motos combustion motocicletas bicicletas patin...,"[0.0765968234597038, -0.1985690365886003, -0.0..."
8,Auteco SAS,ensambladora distribuidora comercializadora mo...,"[0.05453184631800181, -0.21894917425772412, -0..."
9,Comfama - Caja de Compensación Familia de Anti...,servicios financieros habitat educacion cultur...,"[0.08085319259293737, -0.17443099893778227, -0..."


# user word-embeddings

In [15]:
# loop over the company text and return the vector
data_users['vector_u'] = data_users.completo.apply(lambda x : sentence_to_avg(x))

display(data_users).head(10)

,Unnamed: 0,user_id,completo,vector_u
0,1,24785,ciencias sociales hum encantaria tener casa pr...,"[0.09466738636693614, -0.12839831948864477, -0..."
1,2,37144,ciencias sociales hum explorar rincones mundo ...,"[0.11997106335539785, -0.20392183138799197, -0..."
2,3,26147,ciencias sociales hum piloto volando mu...,"[0.07746152904022623, -0.17261421645741426, -0..."
3,4,28094,ciencias aplicadas jefe procesos produccion cu...,"[0.07229771660458223, -0.21662225267466376, -0..."
4,5,34960,ciencias aplicadas crecer ambito profesional p...,"[0.08256272196022284, -0.2114404286079963, 0.0..."
5,6,24411,ciencias aplicadas crear red mundial empresari...,"[0.11033603006683923, -0.20477658384183053, 0...."
6,7,28121,salud httpswwwcomfamacomwebiniciodefaultasp ht...,"[0.08308233472730511, -0.16518969847109272, -0..."
7,8,26534,artes crear organizacion artes escenicas solid...,"[0.05837817945863138, -0.20158419661389984, -0..."
8,9,38173,ciencias aplicadas quiero ser parte empresa si...,"[0.1060779258415739, -0.1705216976285844, -0.0..."
9,10,28432,ciencias sociales hum reinventar evolucionar b...,"[0.15611571952551337, -0.17245859064665314, -0..."


AttributeError: 'NoneType' object has no attribute 'head'

# MATCH BETWEEN USER AND COMPANY

The match is computed by the use of the euclidean distance between the embedding vectors. So that the mininmun distance between them indicates the match.

In [16]:
def euliden_distace(x1,x2):
    dist = math.sqrt((x1-x2)**2)
    return(dist)

def cosine_distace(x1,x2):
    dist_co = dot(x1, x2)/(norm(x1)*norm(x2)) 
    return(dist_co)

In [17]:
cosine_similarity([[1, 0, -1, 5]], [[-1,-1, 0, 4]])

array([[0.8618575]])

In [19]:
df = pd.DataFrame(M)
df['company'] = data_companies2['name']
df.columns = data_users.user_id.tolist() + ['company']
display(df)

,24785,37144,26147,28094,34960,24411,28121,26534,38173,28432,...,32747,25569,28399,24361,6804,24027,6003,35976,36259,company
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Alianza Team®
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Sura
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Interacpedia
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Bancolombia
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Emtelco
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Konecta
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Satrack INC de Colombia Servisat
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Auteco Mobility
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Auteco SAS
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Comfama - Caja de Compensación Familia de Anti...


In [20]:
for i in range(0,len(data_users)):   #
    for j in range(0,len(data_companies2)):  #
        #RESHAPE THE VECTORS
        V1 = data_users.vector_u[i].reshape(1, -1)
        V2 = data_companies2.vector_c[j].reshape(1, -1) 
        # COMPUTE THE COSINE DISTANCE
        df.iloc[j,i] = np.asscalar(cosine_similarity(V1,V2))
             

In [21]:
display(df) 

,24785,37144,26147,28094,34960,24411,28121,26534,38173,28432,...,32747,25569,28399,24361,6804,24027,6003,35976,36259,company
0,0.956311,0.957228,0.917844,0.958316,0.958471,0.952115,0.958343,0.956633,0.934413,0.964050,...,0.829405,0.966318,0.959919,0.960695,0.699691,0.942218,0.794490,0.965799,0.951685,Alianza Team®
1,0.930968,0.949189,0.895850,0.930533,0.931454,0.967351,0.939821,0.928784,0.906696,0.957687,...,0.804988,0.948567,0.960350,0.933436,0.659829,0.908152,0.791988,0.958541,0.944306,Sura
2,0.942278,0.940652,0.897869,0.930333,0.933657,0.943438,0.931744,0.935856,0.891972,0.949177,...,0.815674,0.952408,0.939716,0.929773,0.724496,0.927239,0.790350,0.950648,0.933738,Interacpedia
3,0.945632,0.947791,0.900293,0.941834,0.942440,0.958237,0.942066,0.944120,0.923205,0.960495,...,0.805908,0.958489,0.958936,0.944249,0.693625,0.928802,0.803021,0.960720,0.953459,Bancolombia
4,0.956771,0.957805,0.914607,0.955377,0.956555,0.959611,0.956239,0.955510,0.933530,0.966740,...,0.817765,0.960250,0.962593,0.958026,0.700456,0.943631,0.811175,0.966233,0.960502,Emtelco
5,0.959741,0.963544,0.919122,0.961235,0.960928,0.961350,0.961349,0.962519,0.930902,0.968917,...,0.825715,0.961640,0.959399,0.959927,0.696530,0.945841,0.806588,0.965547,0.959760,Konecta
6,0.940143,0.947778,0.904140,0.942712,0.939628,0.958616,0.943558,0.939079,0.911725,0.951695,...,0.807852,0.956267,0.958413,0.949521,0.701754,0.930934,0.825783,0.961297,0.948851,Satrack INC de Colombia Servisat
7,0.954361,0.965106,0.911707,0.953657,0.958240,0.970118,0.954892,0.959188,0.927076,0.966717,...,0.828346,0.963868,0.969480,0.959128,0.676056,0.936018,0.802936,0.973695,0.962337,Auteco Mobility
8,0.954385,0.959524,0.909034,0.965160,0.963752,0.956636,0.959799,0.965426,0.931533,0.958021,...,0.823225,0.960775,0.954142,0.962990,0.680968,0.939663,0.787528,0.961317,0.948989,Auteco SAS
9,0.950891,0.960133,0.918487,0.952239,0.945425,0.968017,0.954931,0.949161,0.927394,0.972610,...,0.819852,0.960505,0.962188,0.943451,0.674534,0.934034,0.784593,0.962530,0.951868,Comfama - Caja de Compensación Familia de Anti...


# creating a recomendation system by the cosine similarity

In [72]:
#crate an empy data frame
df_match = pd.DataFrame(columns=['user_id','company'], index = data_users['user_id'].tolist())

In [116]:
#case 1: match for the minimum distance -> the most similarity vectors
for col in (df.columns[:-1]):
    maximo = df[col].max()  
    df_match.loc[col,'user_id'] = col
    #df_match.loc[col, 'score'] = maximo
    df_match.loc[col,'company'] = (df[df[col] == maximo]['company'].tolist()[0])
    #print(col,maximo,(df[df[col] == maximo]['company'].tolist()[0]))

In [67]:
df_match.tail(10)

,user_id,company
32747,32747,Alianza Team®
25569,25569,Alianza Team®
28399,28399,Auteco Mobility
24361,24361,Auteco SAS
6804,6804,Interacpedia
24027,24027,Konecta
6003,6003,Satrack INC de Colombia Servisat
35976,35976,Auteco Mobility
36259,36259,Auteco Mobility
company,company,Sura


In [123]:
#case 2: macth using the max and the mean
for col in (df.columns[:-1]):
    maximo = df[col].max()  
    promedio = df[col].mean()  
    df_match.loc[col,'user_id'] = col
    #df_match.loc[col, 'score'] = maximo
    #print(col,df[(df[col]<=maximo)&(df[col] > promedio)]['company'].tolist())
    df_match.loc[col,'company'] = str(df[(df[col]<=maximo)&(df[col] >= promedio)].sort_values(by=col,ascending = False)['company'].tolist())
    #print(col,df[(df[col]<=maximo)&(df[col] > promedio)]['company'].tolist())

In [124]:
df_match

,user_id,company
24785,24785,"['Konecta', 'Emtelco', 'Alianza Team®', 'Autec..."
37144,37144,"['Auteco Mobility', 'Konecta', 'Comfama - Caja..."
26147,26147,"['Konecta', 'Comfama - Caja de Compensación Fa..."
28094,28094,"['Auteco SAS ', 'Konecta', 'Alianza Team®', 'E..."
34960,34960,"['Auteco SAS ', 'Konecta', 'Alianza Team®', 'A..."
24411,24411,"['Auteco Mobility', 'Comfama - Caja de Compens..."
28121,28121,"['Konecta', 'Auteco SAS ', 'Alianza Team®', 'E..."
26534,26534,"['Auteco SAS ', 'Konecta', 'Auteco Mobility', ..."
38173,38173,"['Alianza Team®', 'Emtelco', 'Auteco SAS ', 'K..."
28432,28432,['Comfama - Caja de Compensación Familia de An...


In [108]:
#convert the company colmn to a string
df_match.company[28368]

"['Auteco Mobility', 'Comfama - Caja de Compensación Familia de Antioquia', 'Konecta', 'Alianza Team®', 'Emtelco', 'Auteco SAS ', 'Bancolombia']"

# WRITING THE RESULTS IN THE DATA BASE

create the data in SQL server. Ejecute this query in SQL:

CREATE TABLE match(id INT identity primary key,

[user_id] VARCHAR(255)

,[company] text)

In [109]:
# save the results to the data base
import pyodbc 
import platform 

driver = ''

if platform.system() == 'Windows':
    driver = '{SQL Server}'
else:
    driver = 'ODBC Driver 17 for SQL Server'
            
conection = pyodbc.connect(
    'DRIVER=' + driver + 
    ';SERVER=')

cursor = conection.cursor()


for index,row in df_match.iterrows():
    cursor.execute("INSERT INTO dbo.match([user_id],[company])values (?,?)", row['user_id'],row['company']) 
    
conection.commit()
cursor.close()
conection.close()

